<a href="https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Arboles/Clase_02_Arboles/02_B_Preparando_Datos_para_%C3%81rboles_de_Decisi%C3%B3n_Sin_Resolver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 INTRO: Árboles de Decisión
Explicación de cómo preparar los datos para un modelo de árboles de decisión.

Notebook por [Javier Blanco Cordero](https://www.linkedin.com/in/javier-blanco-cordero-71373656/).

### Enlaces de interés
*   [Slides de presentación](https://docs.google.com/presentation/d/1kiEbdMHy7Ji02SlTxzq913bZ-rcQWn00Td0K_MNVXEk/edit?usp=sharing)
*   [Enlace a este notebook en Colab](https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Arboles/Clase_02_Arboles/02_A_%C3%81rboles_Clasificaci%C3%B3n_M%C3%BAltiple_Resuelto.ipynb)




## 0101 Qué es un árbol de decisión?
Un tipo de algoritmo de aprendizaje supervisado que se basa en realizar particiones a partir de distintos niveles de las variables disponibles.

Los árboles de decisión de scikit learn requiren:


*   Que no haya nulos entre las variables
*   Que todas las variables sean numéricas (hay que dumificar las categóricas)



## 0102 Import
Importamos todas las librerías necesarias para este análisis ([¿No sabes lo que es una librería de Python?](https://www.quora.com/What-is-a-Python-library-and-what-can-I-use-it-for)): pandas, numpy, seaborn, matplotlib.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz 
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

## 0103 Carga el dataset: notas alumnos portugueses
Para probar a transformar datos para entrenar árboles de decisión el dataset sobre las notas finales de ciertos estudiantes portugueses de dos instituos en función de ciertas características socio-demográficas. El dataset original se puede encontrr en Kaggle ([aquí](https://www.kaggle.com/dipam7/student-grade-prediction)).

Esta vez no accederemos a la versión que yo había limpiado previamente, sino al dataset en su formato original, que podéis encontrar listo para importar en mi github: 'https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/Notas_Estudiantes.csv'. 

Importa este dataset en un dataframe llamado **df**.

In [ ]:
# Url archivo raw
url =  'https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/Notas_Estudiantes.csv'

# Importa csv
df = pd.read_csv(url)

# Visualización primeras filas
df.head()

# 02 EDA
Realizaremos un pequeño análisis exploratorio visual para familiarizarnos con el dataset. 

Recuerda que puedes encontrar mis clases sobre análisis exploratorio [aquí](https://github.com/JotaBlanco/TheValley/tree/main/EDA/).

In [ ]:
df.info()

## 0201 Ensuciamos con nulos
Para probar cómo arreglar los nulos, vamos a meter primero ciertos nulos en nuestro dataframe.

In [ ]:
df.columns[:-3]

In [ ]:
df_2 = df.copy(deep=True)

prop_nulos = 0.05
for col in df.columns[:-3]:
  indice_para_nulos = df_2.sample(frac=prop_nulos, replace=False).index
  df_2.loc[indice_para_nulos,col] = np.NaN

df_2.head(10)

In [ ]:
df_2.info()

## 0202 Continuamos con el EDA

In [ ]:
df_2.describe()

In [ ]:
# Visualización coeficientes Pearson
plt.figure(figsize=(12,9))
sns.heatmap(np.round(df_2.corr(),2), 
            vmin=-1, vmax=1, 
            annot=True, cmap="coolwarm")
plt.show()

# 03 Preparación de datos para árbol de decisión
Los árboles de decisión de scikit learn requiren:

*   Que no haya nulos entre las variables
*   Que todas las variables sean numéricas (hay que dumificar las categóricas)


## 0301 Inicialización de árbol
Recordemos cómo entrenar un árbol cuando las variables son numéricas (G2 y G3).

In [ ]:
X = df_2[['G1','G2']]
y = df_2['G3']

display(len(X))
display(X.isna().sum())

In [ ]:
# Inicializo un árbol
arbolito = tree.DecisionTreeRegressor(max_depth=3)
# Entreno el árbol
arbolito = arbolito.fit(X=X, y=y)
# Uso el árbol para predecir y
pred_arbolito = arbolito.predict(X)
# Cómo de buena es la predicción?
mean_squared_error(y, pred_arbolito, squared=False)

In [ ]:
# Con el método export_text en un print()
esquema_print = tree.export_text(arbolito, 
                                 feature_names=list(X.columns))
print(esquema_print)

## 0301 Limpieza de nulos

### 030101 Qué pasa cuando hay nulos en los datos

In [ ]:
df_2[['absences','G1', 'G2', 'G3']].head()

In [ ]:
X = df_2[['absences','G1','G2']]
y = df_2['G3']

display(len(X))
display(X.isna().sum())

In [ ]:
# Inicializo un árbol
arbolito = tree.DecisionTreeRegressor(max_depth=3)
# Entreno el árbol
arbolito = arbolito.fit(X=X, y=y)
# Uso el árbol para predecir y
pred_arbolito = arbolito.predict(X)
# Cómo de buena es la predicción?
mean_squared_error(y, pred_arbolito, squared=False)

### 030202 Eliminando nulos
Con el método .[dropna()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html).

In [ ]:
X = df_2.dropna(subset=['absences'])[['absences','G1','G2']]
y = df_2.dropna(subset=['absences'])['G3']

display(len(X))
display(X.isna().sum())

In [ ]:
# Inicializo un árbol
arbolito = tree.DecisionTreeRegressor(max_depth=3)
# Entreno el árbol
arbolito = arbolito.fit(X=X, y=y)
# Uso el árbol para predecir y
pred_arbolito = arbolito.predict(X)
# Cómo de buena es la predicción?
mean_squared_error(y, pred_arbolito, squared=False)

In [ ]:
# Con el método export_text en un print()
esquema_print = tree.export_text(arbolito, 
                                 feature_names=list(X.columns))
print(esquema_print)

**Problemas de este enfoque**

Eliminando filas tenemos varios problemas: 
*   Perdemos información al perder filas

Sí, nuestra variable estaba a nulo en la filas que hemos eliminado, pero quizás había información útil en el resto de variables que también nos estamos perdiendo.
*   No hemos resuelto el problema de los nulos


Cuando queramos hacer predicciones sobre datos nuevos que contengan nulos tendremos igualmente que desarrollar un método para tratarlos si queremos producir una predicción.



### 030103 Rellenando nulos
Usando el método .[fillna()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html).

#### Rellenando con 0s

In [ ]:
X = df_2.loc[:,['absences','G1','G2']]
X['absences'] = X['absences'].fillna(0)
y = df_2['G3']

display(len(X))
display(X.isna().sum())

In [ ]:
# Inicializo un árbol
arbolito = tree.DecisionTreeRegressor(max_depth=3)
# Entreno el árbol
arbolito = arbolito.fit(X=X, y=y)
# Uso el árbol para predecir y
pred_arbolito = arbolito.predict(X)
# Cómo de buena es la predicción?
mean_squared_error(y, pred_arbolito, squared=False)

In [ ]:
# Con el método export_text en un print()
esquema_print = tree.export_text(arbolito, 
                                 feature_names=list(X.columns))
print(esquema_print)

#### Rellenando con mediana

In [ ]:
X = df_2.loc[:,['absences','G1','G2']]
X['absences'] = X['absences'].fillna(X['absences'].median())
y = df_2['G3']

display(len(X))
display(X.isna().sum())

In [ ]:
# Inicializo un árbol
arbolito = tree.DecisionTreeRegressor(max_depth=3)
# Entreno el árbol
arbolito = arbolito.fit(X=X, y=y)
# Uso el árbol para predecir y
pred_arbolito = arbolito.predict(X)
# Cómo de buena es la predicción?
mean_squared_error(y, pred_arbolito, squared=False)

In [ ]:
# Con el método export_text en un print()
esquema_print = tree.export_text(arbolito, 
                                 feature_names=list(X.columns))
print(esquema_print)

#### Reflexión sobre estos dos últimos árboles
Los dos son iguales! 

Pero el método de relleno de nulos para "absences" provocaba que esos 20 alumnos acabaran a un lado o a otro de la segunda partición en la rama G2<=10.5.

El método de relleno va a influír en dónde caen las filas con nulos cuando hay una partición, por eso es importante y deberá seguir una lógica lo más realista posible.

### 030104 EJERCICIO
Queremos hacer un árbol que prediga la nota 'G3' con las variables ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences'].

Estas variables tienen nulos. Haz 3 modelos:


1.   Un modelo eliminando los nulos
2.   Un modelo rellenando los nulos con 0s
3.   Un modelo rellenando los nulos con un método un poco más avanzado, de tu elección.



In [ ]:
# Visualizamos los datos que vamos a usar
cols = ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 
        'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
df_2[cols].head()

#### Modelo 1
Eliminando los nulos

In [ ]:
# Genera las variables X e Y
cols =  ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 
         'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

X = ...
y = ...

# Visualiza la longitud y el número de nulos
...
...

In [ ]:
# Inicializo un árbol
...
# Entreno el árbol
...
# Uso el árbol para predecir y
...
# Cómo de buena es la predicción?
...

In [ ]:
# Visualiza el árbol
...

#### Modelo 2
Rellenamos los nulos con 0s.

In [ ]:
# Genera las variables X e Y
cols =  ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 
         'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

X = ...
...
y = ...

# Visualiza la longitud y el número de nulos
...
...

In [ ]:
# Inicializo un árbol
...
# Entreno el árbol
...
# Uso el árbol para predecir y
...
# Cómo de buena es la predicción?
...

In [ ]:
# Visualiza el árbol
...

#### Modelo 3
Rellenamos los nulos.

In [ ]:
# Genera las variables X e Y
cols =  ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 
         'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

X = ...
...
...
y = ...

In [ ]:
# Inicializo un árbol
...
# Entreno el árbol
...
# Uso el árbol para predecir y
...
# Cómo de buena es la predicción?
...

In [ ]:
# Visualiza el árbol
...

## 0302 Dumificación vars categóricas

### 030201 Qué pasa cuando hay vars categóricas

In [ ]:
df_2[['school', 'sex', 'Mjob', 'G3']].head()

In [ ]:
X = df_2[['school','sex', 'Mjob']]
y = df_2['G3']

display(len(X))
display(X.isna().sum())

In [ ]:
# Inicializo un árbol
arbolito = tree.DecisionTreeRegressor(max_depth=3)
# Entreno el árbol
arbolito = arbolito.fit(X=X, y=y)
# Uso el árbol para predecir y
pred_arbolito = arbolito.predict(X)
# Cómo de buena es la predicción?
mean_squared_error(y, pred_arbolito, squared=False)

### 030202 Dumificación
Utilizando la función [.get_dummies()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html).

In [ ]:
X.head()

In [ ]:
pd.get_dummies(X, prefix_sep='_').head()

In [ ]:
pd.get_dummies(X, prefix_sep='__', dummy_na=True).head()

In [ ]:
pd.get_dummies(X, prefix_sep='_', drop_first=True).head()

In [ ]:
pd.get_dummies(X, prefix_sep='_', drop_first=True, columns=['school']).head()

In [ ]:
X = pd.get_dummies(df_2[['school','sex', 'Mjob']], 
                   prefix_sep='_', 
                   drop_first=True)
y = df_2['G3']

display(len(X))
display(X.isna().sum())

In [ ]:
# Inicializo un árbol
arbolito = tree.DecisionTreeRegressor(max_depth=4)
# Entreno el árbol
arbolito = arbolito.fit(X=X, y=y)
# Uso el árbol para predecir y
pred_arbolito = arbolito.predict(X)
# Cómo de buena es la predicción?
mean_squared_error(y, pred_arbolito, squared=False)

In [ ]:
# Con el método export_text en un print()
esquema_print = tree.export_text(arbolito, 
                                 feature_names=list(X.columns))
print(esquema_print)

# 04 Ejercicio
Utilizando df_2 (la versión del dataset con nulos), crea un modelo que prediga 'G3' utilizando las variables predictivas:

['school', 'sex', 'age', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'studytime', 'failures', 'schoolsup', 'health', 'absences']



In [ ]:
# Fijate en este subset de datos
df_2[['school',  'Mjob', 'age']].head()

In [ ]:
# Y ahora fíjate en lo que hace esto a partir de dicho subset:
pd.get_dummies(df_2[['school',  'Mjob', 'age']], 
               prefix_sep='_', 
               drop_first=True, 
               columns=['school', 'Mjob'])

In [ ]:
# Ahora genera el dataset X con las variables predictoras dumificadas
X = pd.get_dummies(...)
y = ...

...
...

In [ ]:
# Limpia los nulos
...
...

display(len(X))
display(X.isna().sum())

In [ ]:
# Inicializo un árbol
...
# Entreno el árbol
...
# Uso el árbol para predecir y
...
# Cómo de buena es la predicción?
...

In [ ]:
# Visualiza el árbol
...